In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

This class was originally seen on https://github.com/pytorch/pytorch/issues/1333 and I am just trying to understand it so that I will then be able to do my own

In [2]:
class CausalConv1d(nn.Conv1d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, dilation=1, groups=1, bias=True):
        super(CausalConv1d, self).__init__( in_channels, out_channels, kernel_size, stride=stride, padding=0,
            dilation=dilation, groups=groups, bias=bias)

        self.left_padding = dilation * (kernel_size - 1)

    def forward(self, input):
#         print("Input: ")
#         print(input)
        
        temp = torch.unsqueeze(input, 0)
#         print("Unsqueeze: ")
#         print(temp)
        
        x = F.pad(temp, (self.left_padding, 0, 0, 0))
#         print("Pad: ")
#         print(x)
        
        #x = torch.squeeze(x, 0)
#         print("Squeeze: ")
#         print(x)
        
#         print("Wheights: ")
#         print(self.weight)
        self.weight = nn.Parameter(torch.Tensor([[[2, 0]]]))
#         print(self.weight)
        
#         print("Bias: ")
#         print(self.bias)
        self.bias = nn.Parameter(torch.Tensor([0]))
#         print(self.bias)
        
        return super(CausalConv1d, self).forward(x)

In [3]:
causal = CausalConv1d(1, 1, 2)
print(causal)

CausalConv1d (1, 1, kernel_size=(2,), stride=(1,))


In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(causal.parameters(), lr=0.001, momentum=0.9)

In [5]:
input = Variable(torch.Tensor([[1, 2, 3, 4]]).type(torch.FloatTensor))
print(input)

target = Variable(torch.Tensor([[0, 2, 4, 6]]).type(torch.FloatTensor))
target = torch.unsqueeze(target, 0)
print(target)

Variable containing:
 1  2  3  4
[torch.FloatTensor of size 1x4]

Variable containing:
(0 ,.,.) = 
  0  2  4  6
[torch.FloatTensor of size 1x1x4]



In [6]:
output = causal(input)
print(output)

Variable containing:
(0 ,.,.) = 
  0  2  4  6
[torch.FloatTensor of size 1x1x4]



In [7]:
# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
# output = causal(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

ValueError: Expected 2 or 4 dimensions (got 3)